In [1]:
import os
import sys
import getpass
from binance.spot import Spot

__file__ = %pwd
loc_list = os.path.abspath(__file__).split(os.sep)
HOME_LOC = os.path.join(os.sep, *loc_list[:-2])
sys.path.append(HOME_LOC)
os.chdir(HOME_LOC)
from CODE.Utils.encrypt import *
from CODE.Utils.encrypt import Encrypted_API_key
from CODE.Utils.encrypt import Encrypted_API_secret
from CODE.Utils.utils import *
from CODE.Utils.indicators import *
from CODE.Utils.normalize import *
from CODE.Runner import *
from CODE.RNN import *


/homes/David_Li/Mega/University_of_Adelaide/Works/Courses/4339_COMP_SCI_7318_Deep_Learning_Fundamentals/Assignment3
/homes/David_Li/Mega/University_of_Adelaide/Works/Courses/4339_COMP_SCI_7318_Deep_Learning_Fundamentals/Assignment3


In [2]:
file_paths = load_and_sort_files(os.path.join(HOME_LOC, "DATA", "RAW", "1d"))

In [3]:
def run_various_models(
    conv_configs,
    rnn_configs,
    num_epochs,
    iter_n,
    save_dir,
    silent=False,
    data_len_list=None,
    batch_size=1,
):
    if data_len_list is None:
        data_len_list = len(file_paths)  # 数据长度列表
    for len_ in data_len_list:
        train_X, train_Y, test_X, test_Y = split_data(
            file_paths[(len(file_paths) - len_ - 1) :],
            train_ratio=(len_ - 10) / len_,
            x_length=58,
            y_length=2,
        )

        num_channels = train_X.shape[-1]  # 通道数，对应于特征数
        output_size = train_Y.shape[-1]  # 输出尺寸

        for conv_config in conv_configs:
            for hidden_size, num_rnn_layers in rnn_configs:
                model_p = {
                    "conv_layers_config": conv_config,
                    "num_channels": num_channels,
                    "output_size": output_size,
                    "data_len": train_X.shape[1],
                    "hidden_size": hidden_size,
                    "num_rnn_layers": num_rnn_layers,
                }

                # 创建模型和训练器
                trainer = GroupedTrainer(
                    model=FlexibleRNN,
                    data=(train_X, test_X, train_Y, test_Y),
                    batch_size=batch_size,
                    model_p=model_p,
                    learning_rate=0.001,
                    weight_decay=0.0,
                )
                # trainer.criterion = nn.MSELoss()

                # 训练模型
                trainer.train(
                    num_epochs,
                    iter_n=iter_n,
                    silent=silent,
                )

                # 保存结果到CSV
                file_name = f"{save_dir}/dataset_len_{len_}_conv_{len(conv_config)}_rnn_{num_rnn_layers}_hidden_{hidden_size}.csv"
                trainer.save_metrics_to_csv(file_name)

In [4]:
# 定义卷积层配置
conv_configs = (
    [
        (128, 3, 2),
        (192, 3, 4),
        (192, 3, 3),
        (256, 3, 1),
    ],
)  #


# RNN层的参数
rnn_configs = [
    (1024, 50),  # 中等大小的隐藏层和层数
]


# 调用函数
run_various_models(
    conv_configs=conv_configs,
    rnn_configs=rnn_configs,
    num_epochs=50,
    iter_n=10,
    silent=True,
    save_dir="/Project/David_Li/Works/Courses/4339_COMP_SCI_7318_Deep_Learning_Fundamentals/Assignment3/DATA/Result",
    data_len_list=[100, 200, 400, 800, 1600, 3200],
    batch_size=1024,
)

100%|██████████| 50/50 [00:20<00:00,  2.39it/s]
